In [4]:
import pandas as pd 
import numpy as np
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk import download

In [5]:
# download('stopwords')
stop_words = stopwords.words('english')

In [6]:
df = pd.read_csv('../data/train.csv')

In [7]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [8]:
df = df.dropna()

In [9]:
df.count()

id              404287
qid1            404287
qid2            404287
question1       404287
question2       404287
is_duplicate    404287
dtype: int64

In [10]:
model = KeyedVectors.load_word2vec_format('../../pretrained_models/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [11]:
def preprocess(sentence):
    return [w for w in sentence.lower().split() if w not in stop_words]

def wmd(row):
    q1 = preprocess(row['question1'])
    q2 = preprocess(row['question2'])
    return model.wmdistance(q1, q2)

In [ ]:
df['wmd'] = df.apply(wmd, axis=1)

In [ ]:
inf_df = df[(df == np.inf).any(axis=1)]
non_inf_df = df[~df.isin([np.inf, -np.inf]).any(1)]

In [ ]:
non_inf_df['wmd'].describe()

In [ ]:
duplicates_df = non_inf_df[non_inf_df['is_duplicate']==1]
non__duplicates_df = non_inf_df[non_inf_df['is_duplicate']==0]

In [ ]:
fig = plt.figure(figsize=(10,6))

sns.distplot(duplicates_df['wmd'])
sns.distplot(non__duplicates_df['wmd'])
fig.legend(labels=['duplicates','non duplicates']);

In [ ]:
non__duplicates_df['wmd'].describe()

In [ ]:
duplicates_df['wmd'].describe()

In [ ]:
non__duplicates_df[non__duplicates_df['wmd']==0]

In [ ]:
inf_df.head()